In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!unzip "drive/My Drive/train.zip"
!unzip "drive/My Drive/val.zip"

In [ ]:
import keras,os
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np

# base_dir = '/root/Project'

# train_dir = os.path.join(base_dir, 'train')
# validation_dir = os.path.join(base_dir, 'val')
# test_dir = os.path.join(base_dir, 'test')

train_dir = 'train'
validation_dir = 'val'
test_dir = 'test'

# Directory with our training ad pictures
train_ads_dir = os.path.join(train_dir, 'pos')

# Directory with our training dog pictures
train_noads_dir = os.path.join(train_dir, 'neg')

# Directory with our validation ad pictures
validation_ads_dir = os.path.join(validation_dir, 'pos')

# Directory with our validation dog pictures
validation_noads_dir = os.path.join(validation_dir, 'neg')


In [ ]:
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

In [ ]:
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir, batch_size = 20, class_mode = 'binary', target_size = (300, 300))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory( validation_dir,  batch_size = 20, class_mode = 'binary', target_size = (300, 300))

Found 47057 images belonging to 2 classes.
Found 15685 images belonging to 2 classes.


In [ ]:
from keras.optimizers import Adam
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten , Dropout
from keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape = (300, 300, 3), # Shape of our images
include_top = False, # Leave out the last fully connected layer
weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False
    
# Flatten the output layer to 1 dimension
x = Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = Dropout(0.5)(x)

# Add a final sigmoid layer with 1 node for classification output
x = Dense(1, activation='sigmoid')(x)

model = keras.models.Model(base_model.input, x)

model.compile(optimizer = keras.optimizers.Adam(learning_rate=0.0001), loss = 'binary_crossentropy',metrics = ['acc'])

In [ ]:
vgghist = model.fit(train_generator, validation_data = validation_generator, steps_per_epoch = 100, epochs = 10)

In [ ]:
!unzip "drive/My Drive/test.zip"

In [ ]:
from keras.models import load_model

# Save model
model.save("Ad_NoAd_VGG.h5")

In [ ]:
from keras.models import load_model
from PIL import Image

Image.MAX_IMAGE_PIXELS = None

# Flow test images in batches of 20 using test_datagen generator
test_gen = ImageDataGenerator( rescale = 1.0/255. )
test_generator = test_gen.flow_from_directory( test_dir,  batch_size = 20, class_mode = 'binary', target_size = (300, 300))
saved_model = load_model("Ad_NoAd_VGG.h5")

# Evaluate on test data
scores = saved_model.evaluate(test_generator)
print("%s%s: %.2f%%" % ("evaluate ",saved_model.metrics_names[1], scores[1]*100))

In [ ]:
from sklearn.metrics import confusion_matrix

# y_pred = saved_model.predict(test_generator)
y_pred = saved_model.predict(test_generator)

In [ ]:
y_pred = np.rint(y_pred)
print(confusion_matrix(test_generator.classes, y_pred))
tn, fp, fn, tp = confusion_matrix(test_generator.classes, y_pred).ravel()
print(tn, fp, fn, tp)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(test_generator.classes, y_pred))